In [3]:
# DONE:
# Downloaded both datasets from Desktop
# Removed unnessary columns from the json that showcases retweet and favorite count (interaction_json)
# Merged the interaction_json with questions_json on tweet_id column


# TO DO:
# WARNING - WE CAN MOVE OUR SCHEDULED PRESENTATION TIME TO OFFICE HOURS
    # We can honestly do the presentation itself next week if it sucks
# Take a look at the ReadMe file given with the dataset, it gives alot of important information we can use later.
# Try and find a way to get the raw data as from a url instead of pulling directly from Desktop
    #I tried my best but I couldn't figure it out
# Now that we are here, we can utilize the 'data' Dataframe to answer cool questions such as:
    # Do tweets with alot 
# Potentially use the normal Twitter API for more info/ask more questions
# Potentially use the module 9 notes to do some cool visualizations

In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import json

# Download the data
json_data = pd.read_json('/Users/schned/Desktop/covid19_infodemic_english_data.json',lines=True)


# Removing uncessary columns
interaction_data = json_data.drop(columns=[
 'id_str',
 'full_text',
 'truncated',
 'display_text_range',
 'entities',
 'source',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'user',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'is_quote_status',
 'lang',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'quoted_status',
 'possibly_sensitive',
 'extended_entities','favorited','retweeted'])

# Rename tweet id to use as merge index
interaction_data = interaction_data.rename(columns={'id': 'tweet_id'})

interaction_data

# json_data['favorited'].unique()

,created_at,tweet_id,retweet_count,favorite_count
0,2020-03-20 15:35:29+00:00,1241025578527903750,187425,954568
1,2020-03-19 02:36:13+00:00,1240467080954228737,333831,1505405
2,2020-03-19 19:08:52+00:00,1240716889162018816,160323,660762
3,2020-03-20 18:02:16+00:00,1241062514886090754,154086,368883
4,2020-03-21 19:30:08+00:00,1241447017945223169,126506,445713
...,...,...,...,...
486,2020-03-06 07:35:56+00:00,1235831463121059840,143,461
487,2020-03-06 16:35:12+00:00,1235967176475111432,168,278
488,2020-03-06 22:28:55+00:00,1236056191324864515,125,176
489,2020-03-06 17:54:16+00:00,1235987075440283648,179,539


In [5]:
# Download the other dataset containing the questions
question_data = pd.read_csv('/Users/schned/Desktop/covid19_infodemic_english_data.tsv', delimiter='\t')

# Merge the interaction data table with the question data table on shared 'tweet_id' index.
data = interaction_data.merge(question_data, how='inner', on='tweet_id')

data

,created_at,tweet_id,retweet_count,favorite_count,text,q1_label,q2_label,q3_label,q4_label,q5_label,q6_label,q7_label
0,2020-03-20 15:35:29+00:00,1241025578527903750,187425,954568,For the average American the best way to tell ...,no,NaN,NaN,NaN,NaN,no_joke_or_sarcasm,no_not_interesting
1,2020-03-19 02:36:13+00:00,1240467080954228737,333831,1505405,this is fucking bullshit,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
2,2020-03-19 19:08:52+00:00,1240716889162018816,160323,660762,Can y’all please just follow the government’s ...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
3,2020-03-20 18:02:16+00:00,1241062514886090754,154086,368883,No offense but the corona virus disappearing b...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
4,2020-03-21 19:30:08+00:00,1241447017945223169,126506,445713,This is the face of someone who just spent 9 h...,yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,1_no_definitely_not_harmful,yes_not_urgent,no_not_harmful,no_not_interesting
...,...,...,...,...,...,...,...,...,...,...,...,...
486,2020-03-06 07:35:56+00:00,1235831463121059840,143,461,For example: When Trump invents numbers and ig...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
487,2020-03-06 16:35:12+00:00,1235967176475111432,168,278,President Trump's comments about the coronavir...,yes,4_yes_probably_contains_false_info,4_yes_probably_of_interest,5_yes_definitely_harmful,yes_not_urgent,yes_rumor_conspiracy,no_not_interesting
488,2020-03-06 22:28:55+00:00,1236056191324864515,125,176,Drug companies reportedly killed a provision i...,yes,1_no_definitely_contains_no_false_info,4_yes_probably_of_interest,5_yes_definitely_harmful,yes_not_urgent,no_not_harmful,yes_blame_authorities
489,2020-03-06 17:54:16+00:00,1235987075440283648,179,539,Does @twitter have a team fighting covid-19 ru...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting


,created_at,tweet_id,retweet_count,favorite_count,text,q1_label,q2_label,q3_label,q4_label,q5_label,q6_label,q7_label
0,2020-03-20 15:35:29+00:00,1241025578527903750,187425,954568,For the average American the best way to tell ...,no,NaN,NaN,NaN,NaN,no_joke_or_sarcasm,no_not_interesting
1,2020-03-19 02:36:13+00:00,1240467080954228737,333831,1505405,this is fucking bullshit,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
2,2020-03-19 19:08:52+00:00,1240716889162018816,160323,660762,Can y’all please just follow the government’s ...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
3,2020-03-20 18:02:16+00:00,1241062514886090754,154086,368883,No offense but the corona virus disappearing b...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
4,2020-03-21 19:30:08+00:00,1241447017945223169,126506,445713,This is the face of someone who just spent 9 h...,yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,1_no_definitely_not_harmful,yes_not_urgent,no_not_harmful,no_not_interesting
...,...,...,...,...,...,...,...,...,...,...,...,...
486,2020-03-06 07:35:56+00:00,1235831463121059840,143,461,For example: When Trump invents numbers and ig...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
487,2020-03-06 16:35:12+00:00,1235967176475111432,168,278,President Trump's comments about the coronavir...,yes,4_yes_probably_contains_false_info,4_yes_probably_of_interest,5_yes_definitely_harmful,yes_not_urgent,yes_rumor_conspiracy,no_not_interesting
488,2020-03-06 22:28:55+00:00,1236056191324864515,125,176,Drug companies reportedly killed a provision i...,yes,1_no_definitely_contains_no_false_info,4_yes_probably_of_interest,5_yes_definitely_harmful,yes_not_urgent,no_not_harmful,yes_blame_authorities
489,2020-03-06 17:54:16+00:00,1235987075440283648,179,539,Does @twitter have a team fighting covid-19 ru...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
